# Imports

In [1]:
import typing
import json
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import os
import os.path
import numpy as np
from tensorflow import keras
import csv
import ipywidgets
from IPython.display import display
import skimage
import skimage.feature
import math
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

2023-05-24 08:53:41.829284: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data

## Emoji

In [2]:
EMOJI_PATH = "./data/emoji/"

- 0 = Sad
- 1 = Happy

### Name to emoticon

In [3]:
happy_emojis = [
    "GRINNING FACE",
    "GRINNING FACE WITH SMILING EYES",
    "ROLLING ON THE FLOOR LAUGHING",
    "FACE WITH TEARS OF JOY",
    "SLIGHTLY SMILING FACE",
    "UPSIDE-DOWN FACE",
    "MELTING FACE",
    "WINKING FACE",
    "SMILING FACE WITH SMILING EYES",
    "SMILING FACE WITH HALO",
    "KISSING FACE",
    "KISSING FACE WITH CLOSED EYES",
    "KISSING FACE WITH SMILING EYES",
    "SMILING FACE WITH TEAR",
    "MONEY-MOUTH FACE",
    "SMIRKING FACE",
    "RELIEVED FACE",
    "DROOLING FACE",
    "SMILING FACE WITH SUNGLASSES",
    "NERD FACE",
    "FACE HOLDING BACK TEARS",
    "SMILING FACE WITH HORNS",
    "GRINNING FACE WITH SMILING EYES",
    "CAT FACE WITH TEARS OF JOY",
    "CAT FACE WITH WRY SMILE",
    "FACE WITH COWBOY HAT",
    "FACE THROWING A KISS",
    "FACE SAVOURING DELICIOUS FOOD",
    "SMILING FACE WITH SMILING EYES AND HAND COVERING MOUTH",
    "FACE WITH STUCK-OUT TONGUE",
    "SMILING CAT FACE WITH OPEN MOUTH",
    "GRINNING CAT FACE WITH SMILING EYES",
    "SMILING FACE WITH OPEN MOUTH",
    "SMILING FACE WITH OPEN MOUTH AND COLD SWEAT",
    "SMILING FACE WITH OPEN MOUTH AND TIGHTLY-CLOSED EYES",
    "KISSING CAT FACE WITH CLOSED EYES",
    "FACE WITH PARTY HORN AND PARTY HAT",
    "SMILING CAT FACE WITH HEART-SHAPED EYES",
    "WHITE SMILING FACE",
    "SMILING FACE WITH HEART-SHAPED EYES",
    "SMILING FACE WITH SMILING EYES AND THREE HEARTS",
    "HUGGING FACE",
    "FACE WITH STUCK-OUT TONGUE AND TIGHTLY-CLOSED EYES",
    "GRINNING FACE WITH STAR EYES",
    "FACE WITH STUCK-OUT TONGUE AND WINKING EYE",
    "GRINNING FACE WITH ONE LARGE AND ONE SMALL EYE",
]

sad_emojis = [
    "UNAMUSED FACE",
    "FACE WITH ROLLING EYES",
    "GRIMACING FACE",
    "SHAKING FACE",
    "FACE EXHALING",
    "LYING FACE",
    "PENSIVE FACE",
    "SLEEPY FACE",
    "FACE WITH THERMOMETER",
    "FACE WITH HEAD-BANDAGE",
    "NAUSEATED FACE",
    "SNEEZING FACE",
    "DIZZY FACE",
    "FACE WITH SPIRAL EYES",
    "CONFUSED FACE",
    "WORRIED FACE",
    "SLIGHTLY FROWNING FACE",
    "FROWNING FACE",
    "FACE WITH OPEN MOUTH",
    "HUSHED FACE",
    "ASTONISHED FACE",
    "FROWNING FACE WITH OPEN MOUTH",
    "ANGUISHED FACE",
    "FEARFUL FACE",
    "DISAPPOINTED BUT RELIEVED FACE",
    "CRYING FACE",
    "LOUDLY CRYING FACE",
    "FACE SCREAMING IN FEAR",
    "CONFOUNDED FACE",
    "PERSEVERING FACE",
    "DISAPPOINTED FACE",
    "WEARY FACE",
    "TIRED FACE",
    "ANGRY FACE",
    "WEARY CAT FACE",
    "IMP",
    "FACE WITH OPEN MOUTH AND COLD SWEAT",
    "CRYING CAT FACE",
    "FACE WITH COLD SWEAT",
    "POUTING FACE",
    "FACE WITH OPEN MOUTH VOMITING",
    "FACE WITH LOOK OF TRIUMPH",
    "SERIOUS FACE WITH SYMBOLS COVERING MOUTH",
    "FACE WITH PLEADING EYES",
    "POUTING CAT FACE",
]

name_to_emotion = dict()

for name in happy_emojis:
    name_to_emotion[name] = 1
for name in sad_emojis:
    name_to_emotion[name] = 0

### Rest

In [4]:
emoji_images = []
emoji_emotions = []

with open(os.path.join(EMOJI_PATH, "emoji.json")) as f:
    emoji_json = json.load(f)

for entry in emoji_json:
    if (emotion := name_to_emotion.get(entry["name"])) is None:
        continue
    
    key_path = [
        ("has_img_apple", "img-apple-64"),
        ("has_img_facebook", "img-facebook-64"),
        ("has_img_google", "img-google-64"),
        ("has_img_twitter", "img-twitter-64"),
    ]
    for key, idk_path in key_path:
        if entry[key]:
            path = os.path.join(EMOJI_PATH, idk_path, entry["image"])
            with PIL.Image.open(path) as image:
                image: PIL.Image = image
                image = image.convert("RGBA")
                image = image.resize((48, 48))
                image_array = np.asarray(image)
                
                emoji_images.append(image_array)
                emoji_emotions.append(emotion)
                
emoji_images = np.array(emoji_images)
emoji_emotions = np.array(emoji_emotions)

## Face

- 0 = Angry
- 1 = Disgust
- 2 = Fear
- 3 = Happy
- 4 = Sad
- 5 = Surprise
- 6 = Neutral

In [2]:
FACE_PATH = "./data/face/"

In [3]:
# if False:
if True:
    image_shape = (48, 48)
    
    face_emotions = []

    with open(os.path.join(FACE_PATH, "data.csv"), "rb") as f:
        row_count = (
              sum(1 for _ in f)
            - 1 # subtract one to skip header
        )

    widget_progress = ipywidgets.IntProgress(max=row_count-1)
    display(widget_progress)
    
    face_images = np.zeros((row_count,) + image_shape, dtype=int)
    face_emotions = np.zeros((row_count,), dtype=int)
    face_extended = np.zeros((row_count, 10), dtype=int)

    with (
        open(os.path.join(FACE_PATH, "data.csv")) as f,
        open(os.path.join(FACE_PATH, "data-extended.csv")) as g,
    ):
        f = csv.reader(f)
        g = csv.reader(g)
        _ = next(f)
        _ = next(g)
        for i, (row, row_extended) in enumerate(zip(f, g)):
            # if i >= 500:
            #     break
            
            face_emotions[i] = int(row[0])
            image = np.fromiter((int(val) for val in row[2].split()), int)
            image = np.reshape(image, image_shape)
            face_images[i] = image
            
            for j, v in enumerate(row_extended[2:]):
                face_extended[i,j] = int(v)

            widget_progress.value = i

IntProgress(value=0, max=35886)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



# Solution

## Emoji

In [25]:
test_percent = 0.80

length = len(emoji_images)

# preprocess data
data_in = emoji_images / 255
data_out = np.stack(
    [
        emoji_emotions == 0,
        emoji_emotions == 1,
    ],
    axis=1
).astype(float)

split = np.zeros(length, dtype=int)
split_index = int(length * test_percent)
split[:split_index] = 1

np.random.shuffle(split)

# train model
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 4)))
model.add(keras.layers.MaxPool2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    data_in[split == 1], data_out[split == 1],
    validation_data=(data_in[split == 0], data_out[split == 0]),
    epochs=32,
    batch_size=64,
)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 46, 46, 32)        1184      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 10, 10, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 8, 8, 64)          36928     
                                                                 
 flatten_7 (Flatten)         (None, 4096)             

KeyboardInterrupt: 

## Faces (trained, manual)

In [ ]:
image = face_images[0]
image = skimage.transform.rescale(image, 2, anti_aliasing=True)
fd, hog_image = skimage.feature.hog(image, orientations=8, pixels_per_cell=(8, 8),
                    cells_per_block=(1, 1), visualize=True)

# print(hog_image)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = skimage.exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

In [20]:
if True:
    # extract features
    data_face_features = np.zeros((len(face_images), 36*32))

    widget_progress = ipywidgets.IntProgress(max=len(face_images)-1)
    display(widget_progress)

    for i, image in enumerate(face_images):
        image = skimage.transform.rescale(image, 2, anti_aliasing=True)
        data_face_features[i] = skimage.feature.hog(
            image,
            orientations=8,
            pixels_per_cell=(8, 8),
            cells_per_block=(1, 1),
            feature_vector=True,
        )

        widget_progress.value = i

IntProgress(value=0, max=35886)

In [27]:
data_out = np.stack(
    [
        face_emotions == 0,
        face_emotions == 1,
        face_emotions == 2,
        face_emotions == 3,
        face_emotions == 4,
        face_emotions == 5,
        face_emotions == 6,
    ],
    axis=1,
).astype(float)

In [28]:
print(data_out[1])

[1. 0. 0. 0. 0. 0. 0.]


In [29]:
test_percent = 0.80

length = len(face_images)

# split into train and test
split = np.zeros(length, dtype=int)
split_index = int(length * test_percent)
split[:split_index] = 1

np.random.shuffle(split)

# train model
model = keras.models.Sequential()
# model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(36, 32, 1)))
# model.add(keras.layers.MaxPool2D((2, 2)))
# model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(keras.layers.MaxPool2D((2, 2)))
# model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu', input_shape=(36*32,)))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(7, activation='softmax'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
# model.build()
model.summary()

history = model.fit(
    data_face_features[split == 1], data_out[split == 1],
    validation_data=(data_face_features[split == 0], data_out[split == 0]),
    epochs=12,
    batch_size=64,
)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 128)               147584    
                                                                 
 dense_43 (Dense)            (None, 128)               16512     
                                                                 
 dense_44 (Dense)            (None, 64)                8256      
                                                                 
 dense_45 (Dense)            (None, 32)                2080      
                                                                 
 dense_46 (Dense)            (None, 7)                 231       
                                                                 
Total params: 174,663
Trainable params: 174,663
Non-trainable params: 0
_________________________________________________________________
Epoch 1/12
449/449 [===========================

## Faces (trained, automated, multi-label)

In [31]:
data_in_faces_multi_label = face_images / 255
data_out = (face_extended > 1).astype(float)
# data_out = face_extended.astype(float) / 10

In [32]:
test_percent = 0.80

length = len(face_images)

# split into train and test
split = np.zeros(length, dtype=int)
split_index = int(length * test_percent)
split[:split_index] = 1

np.random.shuffle(split)

# train model
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(keras.layers.MaxPool2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
# model.build()
model.summary()

history = model.fit(
    data_in_faces_multi_label[split == 1], data_out[split == 1],
    validation_data=(data_in_faces_multi_label[split == 0], data_out[split == 0]),
    epochs=10,
    batch_size=64,
)

model.save("model_3")

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 10, 10, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 8, 8, 64)          36928     
                                                                 
 flatten_9 (Flatten)         (None, 4096)            

## Faces

In [26]:
length = 10000

In [27]:
base_model = MobileNetV2(
    input_shape=(48, 48, 3),
    include_top=False,
    weights="imagenet",
    pooling="avg",
)

face_images_local = face_images[:length]
face_images_rgb = np.stack(
    [
        face_images_local,
        face_images_local,
        face_images_local,
    ],
    axis=3,
)

face_embedding = base_model(face_images_rgb)
face_embedding = keras.layers.Flatten()(face_embedding)
face_embedding = keras.layers.Dropout(0.5)(face_embedding)

In [28]:
face_emotions_local = face_emotions[:length]
data_out = np.stack(
    [
        face_emotions_local == 0,
        face_emotions_local == 1,
        face_emotions_local == 2,
        face_emotions_local == 3,
        face_emotions_local == 4,
        face_emotions_local == 5,
        face_emotions_local == 6,
    ],
    axis=1,
).astype(float)

In [34]:
test_percent = 0.80

# split into train and test
split = np.zeros(length, dtype=int)
split_index = int(length * test_percent)
split[:split_index] = 1

np.random.shuffle(split)

# train model
model = keras.models.Sequential()
model.add(keras.layers.Input((1280,)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(7, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
# model.build()
model.summary()

history = model.fit(
    face_embedding[split == 1], data_out[split == 1],
    validation_data=(face_embedding[split == 0], data_out[split == 0]),
    epochs=64,
    batch_size=64,
)

model.save("model_4")

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 64)                81984     
                                                                 
 dense_63 (Dense)            (None, 32)                2080      
                                                                 
 dense_64 (Dense)            (None, 8)                 264       
                                                                 
 dense_65 (Dense)            (None, 8)                 72        
                                                                 
 dense_66 (Dense)            (None, 8)                 72        
                                                                 
 dense_67 (Dense)            (None, 7)                 63        
                                                                 
Total params: 84,535
Trainable params: 84,535
Non-tra